In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import load_model
import os

def calculate_gaze_vector_image(image_path, model):
    image_save_prefix = image_path.split('/')[-1].split('.')[0]
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Resize and normalize
    image_resized = cv2.resize(image, (128, 128))
    image_normalized = image_resized / 255.0

    # Prepare input for model
    input_image = np.expand_dims(image_normalized, axis=0)  # (1, 128, 128)
    input_image = np.expand_dims(input_image, axis=-1)      # (1, 128, 128, 1)

    # Display the mask with circles
    # plt.figure(figsize=(8, 8))
    # plt.imshow(image_resized)
    # plt.title('Predicted Mask with Circles')
    # plt.axis('off')
    # plt.show()


    # Predict mask
    predicted_mask =  model.predict(input_image)[0, :, :, 0]  # (128, 128)

    plt.figure(figsize=(8, 8))
    plt.imshow(predicted_mask)
    plt.title('Predicted Mask with Circles')
    plt.axis('off')
    plt.show()

    #scalar region
    scalar_lower_bound = 0.2
    scalar_upper_bound = 0.5
    scalar_region_mask = np.logical_and(predicted_mask >= scalar_lower_bound, predicted_mask <= scalar_upper_bound).astype(np.uint8)

    iris_lower_bound = 0.75
    iris_upper_bound = 1
    iris_region_mask = np.logical_and(predicted_mask >= iris_lower_bound, predicted_mask <= iris_upper_bound).astype(np.uint8)

    # Find contours from the binary mask
    unfiltered_scalar_contours, _ = cv2.findContours(scalar_region_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    iris_contours, _ = cv2.findContours(iris_region_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(iris_contours) == 0:
        print("No iris contours found.")
        return None

    scalar_contours = [cnt for cnt in unfiltered_scalar_contours if cv2.contourArea(cnt) > 25]

    # Find the iris contour with the largest area
    largest_iris_contour = max(iris_contours, key=cv2.contourArea)

    M = cv2.moments(largest_iris_contour)

    if M["m00"] != 0:
        center_x = int(M["m10"] / M["m00"])
        center_y = int(M["m01"] / M["m00"])
    else:
        # fallback if the area is too small
        center_x, center_y = 0, 0

    iris_center = (center_x, center_y)

    # Initialize leftmost and rightmost points
    leftmost = (128, 128)
    rightmost = (0, 0)

    # Find extreme points from all contours
    for contour in scalar_contours:
        for point in contour:
            x, y = point[0]
            if x < leftmost[0]:
                leftmost = (x, y)
            if x > rightmost[0]:
                rightmost = (x, y)

    print(f"Leftmost Point: {leftmost}")
    print(f"Rightmost Point: {rightmost}")

    scelar_vector =  np.array(rightmost) - np.array(leftmost)
    middle_point = ((leftmost[0] + rightmost[0]) // 2, (leftmost[1] + rightmost[1]) // 2)
    # Convert predicted mask to RGB to draw colored circles
    predicted_mask_rgb = cv2.cvtColor((predicted_mask * 255).astype(np.uint8), cv2.COLOR_GRAY2RGB)

    # Draw circles at leftmost and rightmost points
    cv2.circle(predicted_mask_rgb, leftmost, radius=2, color=(255, 0, 0), thickness=-1)  # Blue circle
    cv2.circle(predicted_mask_rgb, rightmost, radius=2, color=(0, 0, 255), thickness=-1) # Red circle
    cv2.circle(predicted_mask_rgb, iris_center, radius=2, color=(0, 255, 255), thickness=-1) # Red circle
    cv2.circle(predicted_mask_rgb, middle_point, radius=2, color=(225, 0, 255), thickness=-1)  # Magenta circle
    cv2.line(predicted_mask_rgb, leftmost, rightmost, color=(0, 255, 0), thickness=1)
    cv2.arrowedLine(predicted_mask_rgb, middle_point, iris_center, color=(255, 0, 0), thickness=1, tipLength=0.1)

    # Display the mask with circles
    plt.figure(figsize=(8, 8))
    plt.imshow(predicted_mask_rgb)
    plt.title('Predicted Mask with Circles')
    plt.axis('off')
    plt.show()

    # Create the directory if it doesn't exist
    output_dir = "gaze_vector_prediction"
    os.makedirs(output_dir, exist_ok=True)

    # Define the output file path
    output_file_path = os.path.join(output_dir, image_save_prefix+"_vectors.png")

    # Save the image
    cv2.imwrite(output_file_path, cv2.cvtColor(predicted_mask_rgb, cv2.COLOR_RGB2BGR))
    print(f"Image saved to {output_file_path}")

2025-04-29 13:27:12.043532: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-29 13:27:12.058740: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745913432.081780 1186154 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745913432.089055 1186154 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745913432.108025 1186154 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
image_dir_path =  '/home/yasas/GazeEstimation/openEDS/openEDS/test/images'
image_files = sorted(os.listdir(image_dir_path))

def dice_loss(y_true, y_pred):
    smooth = 1.0
    intersection = tf.reduce_sum(y_true * y_pred)
    return 1 - (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

# Now load the model properly
model = load_model('light_weight_unet_gaze_estimation_model.h5', custom_objects={'dice_loss': dice_loss})


for image_file in image_files:
    image_path = os.path.join(image_dir_path, image_file)
    print(image_path)
    print(f"Processing {image_path}...")
    calculate_gaze_vector_image(image_path, model)
